In [1]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 36.4 kB/s eta 0:00:00m eta 0:00:0136m0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.14-py3-none-any.whl size=105120 sha256=be9d865746f710834e428227d96fc642665f9060caef695b2280e1fc4c3cb40b
  Stored in directory: /home/rishi/.cache/pip/wheels/ec/d9/04/93be57edf3835182a8e76d8ff87f7e77a0113f473ba3721ee0
Successfully built kaggle


In [2]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 14.3 kB/s eta 0:00:00 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB ? eta 0:00:00MB/s eta 0:00:010:00:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 26.6 kB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 17.1 kB/s eta 0:00:0031m41.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 484.8 kB/s eta 0:00:00eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 3.0 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 1.7 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 999.8 kB/s eta 0:00:001m18.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━

In [1]:
# Import the modules
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2


2024-06-08 17:11:45.802346: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'cv2'

In [22]:
# Define directories
train_dir = 'data/skin-disease-datasaet/train_set'
validation_dir = 'data/skin-disease-datasaet/test_set'

print(type(train_dir))

# augment the training data to be more random
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation data
validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load training data to the training data generator
train = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize images to a good size
    batch_size=32,
    class_mode='sparse'      
)

# Load validation for the other modle
validation = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),  # Same reason as b4
    batch_size=32,
    class_mode='sparse'      
)


<class 'str'>
Found 4637 images belonging to 28 classes.
Found 1187 images belonging to 28 classes.


In [19]:
# Create the NN
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(train.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model with the train and validation
history = model.fit(
    train,
    steps_per_epoch=train.samples // train.batch_size,
    epochs=25,
    validation_data=validation,
    validation_steps=validation.samples // validation.batch_size
)

# Save the trained model
model.save('skin_disease_model3.h5')

2024-06-08 21:38:22.309273: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


Epoch 1/25
144/144 [==============================] - ETA: 0s - loss: 2.4681 - accuracy: 0.3702

2024-06-08 21:46:41.140692: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


144/144 [==============================] - 633s 4s/step - loss: 2.4681 - accuracy: 0.3702 - val_loss: 2.8615 - val_accuracy: 0.2264
Epoch 2/25
144/144 [==============================] - 1160s 8s/step - loss: 2.2439 - accuracy: 0.4022 - val_loss: 2.7157 - val_accuracy: 0.2416
Epoch 3/25
144/144 [==============================] - 300s 2s/step - loss: 2.1062 - accuracy: 0.4124 - val_loss: 2.4977 - val_accuracy: 0.2694
Epoch 4/25
144/144 [==============================] - 462s 3s/step - loss: 1.9526 - accuracy: 0.4426 - val_loss: 2.6955 - val_accuracy: 0.2922
Epoch 5/25
144/144 [==============================] - 695s 5s/step - loss: 1.9071 - accuracy: 0.4452 - val_loss: 2.4860 - val_accuracy: 0.3100
Epoch 6/25
144/144 [==============================] - 349s 2s/step - loss: 1.8211 - accuracy: 0.4658 - val_loss: 2.6538 - val_accuracy: 0.2981
Epoch 7/25
144/144 [==============================] - 205s 1s/step - loss: 1.7763 - accuracy: 0.4727 - val_loss: 2.7493 - val_accuracy: 0.3302
Epoch 8/2

In [14]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_image(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    predictions = model.predict(img_array)
    return predictions

img_path = 'Nail.jpg'
for i in range(4):
    predictions = predict_image(img_path, model)
    predicted_class = np.argmax(predictions)
    class_labels = list(train_generator.class_indices.keys())
    print(f"Predicted Class: {class_labels[predicted_class]}")


1/1 [==============================] - 0s 49ms/step
Predicted Class: FU-athlete-foot
1/1 [==============================] - 0s 38ms/step
Predicted Class: FU-athlete-foot
1/1 [==============================] - 0s 61ms/step
Predicted Class: FU-athlete-foot
1/1 [==============================] - 0s 57ms/step
Predicted Class: FU-athlete-foot


In [28]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# Load the trained model
model = tf.keras.models.load_model('skin_disease_model2.h5')

# Assuming the class labels are already known (from train_generator)
class_labels = list(train.class_indices.keys())

def predict_top_3(image_path, model):
    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    
    # Make predictions
    predictions = model.predict(img_array)
    
    # Get the top 3 predictions
    top_3_indices = predictions[0].argsort()[-3:][::-1]
    top_3_probs = predictions[0][top_3_indices]
    
    # Get the top 3 class labels
    top_3_labels = [class_labels[i] for i in top_3_indices]
    
    return list(zip(top_3_labels, top_3_probs))

# Example usage
image_path = 'Test.jpg'
top_3_predictions = predict_top_3(image_path, model)

# Print the top 3 predictions
for label, prob in top_3_predictions:
    print(f"{label}: {prob:.2%}")


1/1 [==============================] - 1s 772ms/step
Light Diseases and Disorders of Pigmentation: 31.91%
VI-chickenpox: 28.52%
normal: 19.92%
